# Imports

In [1]:
import pandas as pd, numpy as np
import requests
from urllib.parse import urljoin

from dotenv import dotenv_values, find_dotenv
from crypto_trading.amberdata import get_exchange_reference_futures, get_ohlcv_info_futures

In [2]:
%load_ext autoreload
%autoreload 2

# Config / Params

## Credentials

In [2]:
config = dotenv_values(find_dotenv())

ad_api_key = config.get('AMBERDATA_API_KEY')

## API Params

In [6]:
headers = {
    'accept':'application/json',
    'x-api-key':ad_api_key,
}

ad_base_url = 'https://api.amberdata.com'

In [7]:
endpoint = 'market/futures/ohlcv/information'

# Get Instrument Data

## Get All Symbol Metadata

In [39]:
endpoint_exchange_reference = 'market/futures/exchanges/reference'
url_exchange_reference = urljoin(ad_base_url, endpoint_exchange_reference)

params = {
    'exchange':'binance',
    'includeInactive':'true',
}

with requests.Session() as session:
    session.headers.update(headers)

    list_instruments = []
    while url_exchange_reference is not None:
        response = session.get(url_exchange_reference, params=params)
        payload = response.json().get('payload', {})
        metadata = payload.get('metadata', {})
        data = payload.get('data', {})

        list_instruments.extend(
            pd.DataFrame(
                [{'instrument': instrument, **data_exchange[instrument]} for instrument in data_exchange.keys()]
            ).assign(exchange=exchange)
            for exchange, data_exchange in data.items()
        )

        url_exchange_reference = metadata.get('next', None)

df_exchange_reference = pd.concat(list_instruments, axis=0)

## Get First/Last Data

### Params

In [53]:
endpoint_ohlcv_info = 'market/futures/ohlcv/information'
url_ohlcv_info = urljoin(ad_base_url, endpoint_ohlcv_info)

params = {
    'exchange':'binance',
    'includeInactive':'true',
}

with requests.Session() as session:
    session.headers.update(headers)

    list_instruments = []
    while url_ohlcv_info is not None:
        response = session.get(url_ohlcv_info, params=params)
        payload = response.json().get('payload', {})
        metadata = payload.get('metadata', {})
        data = payload.get('data', {})

        list_instruments.append(pd.DataFrame(data))

        url_ohlcv_info = metadata.get('next', None)

df_ohlcv_info = pd.concat(list_instruments, axis=0)

In [54]:
df_ohlcv_info

,exchange,instrument,startDate,endDate,active
0,binance,1000BONKUSDC,1.714635e+12,1718140560000,True
1,binance,1000BONKUSDT,1.700662e+12,1718140560000,True
2,binance,1000BTTCUSDT,1.643168e+12,1649667600000,False
3,binance,1000FLOKIUSDT,1.683391e+12,1718140560000,True
4,binance,1000LUNCBUSD,1.653965e+12,1686213060000,False
...,...,...,...,...,...
64,binance,ZENUSDT,1.606201e+12,1718140560000,True
65,binance,ZETAUSDT,1.706863e+12,1718140560000,True
66,binance,ZILUSDT,1.592471e+12,1718140560000,True
67,binance,ZILUSD_PERP,1.649302e+12,1683590400000,False


In [48]:
df_exchange_reference.shape

(581, 28)

In [49]:
df_ohlcv_info.shape

(569, 5)

In [50]:
display(df_exchange_reference.head())
display(df_ohlcv_info.head())

,instrument,nativeInstrument,nativeBaseSymbol,nativeQuoteSymbol,baseSymbol,quoteSymbol,underlying,quoteAsset,expiration,active,...,limits_market_max,limits_market_min,limits_price_max,limits_price_min,precision_amount,precision_base,precision_price,precision_quote,settleAsset,exchange
0,1000BONKUSDC,1000BONKUSDC,1000BONK,USDC,1000BONK,USDC,1000BONK,USDC,,True,...,300000,1,200,0.000001,1.0,0.0,0.000001,0.0,None,binance
1,1000BONKUSDT,1000BONKUSDT,1000BONK,USDT,1000BONK,USDT,1000BONK,USDT,,True,...,6000000,1,200,0.000001,1.0,0.0,0.000001,0.0,None,binance
2,1000BTTCUSDT,1000BTTCUSDT,None,None,None,None,1000BTTC,USDT,,False,...,12000000,1,10000,0.00001,1.0,0.0,0.000001,0.0,None,binance
3,1000FLOKIUSDT,1000FLOKIUSDT,1000FLOKI,USDT,1000FLOKI,USDT,1000FLOKI,USDT,,True,...,638000,1,200,0.00001,1.0,0.0,0.000010,0.0,None,binance
4,1000LUNCBUSD,1000LUNCBUSD,1000LUNC,BUSD,1000LUNC,BUSD,1000LUNC,BUSD,,False,...,2000000,1,200,0.0001,1.0,0.0,0.000010,0.0,None,binance


,exchange,instrument,startDate,endDate,active
0,binance,1000BONKUSDC,1.714635e+12,1718140500000,True
1,binance,1000BONKUSDT,1.700662e+12,1718140500000,True
2,binance,1000BTTCUSDT,1.643168e+12,1649667600000,False
3,binance,1000FLOKIUSDT,1.683391e+12,1718140500000,True
4,binance,1000LUNCBUSD,1.653965e+12,1686213060000,False


In [3]:
df_exchange_reference = get_exchange_reference_futures(exchange='binance')
df_ohlcv_info = get_ohlcv_info_futures(exchange='binance')

In [4]:
df_instrument_data_merged = pd.merge(
    df_exchange_reference,
    df_ohlcv_info.drop('active', axis='columns'),
    on=['instrument', 'exchange']
)
# df_instrument_data_merged.loc[df_instrument_data_merged['active_y'].isnull()]

In [7]:
df_instrument_data_merged.iloc[0]

instrument                1000BONKUSDC
nativeInstrument          1000BONKUSDC
nativeBaseSymbol              1000BONK
nativeQuoteSymbol                 USDC
baseSymbol                    1000BONK
quoteSymbol                       USDC
underlying                    1000BONK
quoteAsset                        USDC
expiration                            
active                            True
contractSize                      None
contractType                 perpetual
marginType                      linear
limits_amount_max             10000000
limits_amount_min                    1
limits_cost_max                   None
limits_leverage_max               None
limits_leverage_min               None
limits_market_max               300000
limits_market_min                    1
limits_price_max                   200
limits_price_min              0.000001
precision_amount                   1.0
precision_base                     0.0
precision_price               0.000001
precision_quote          

In [37]:
df_instrument_data_merged.loc[df_instrument_data_merged['startDate'].isnull()]

,instrument,nativeInstrument,nativeBaseSymbol,nativeQuoteSymbol,baseSymbol,quoteSymbol,underlying,quoteAsset,expiration,active,...,limits_price_max,limits_price_min,precision_amount,precision_base,precision_price,precision_quote,settleAsset,exchange,startDate,endDate
323,IOUSDT,IOUSDT,IO,USDT,IO,USDT,IO,USDT,,True,...,2000,0.001,0.1,0.0,0.001,0.0,None,binance,NaN,1718142000000


In [11]:
from datetime import datetime, timezone

In [12]:
datetime.now(timezone.utc).timestamp()

1718148426.762746

In [18]:
datetime(2017, 1, 1).timestamp()

1483246800.0

In [28]:
from numpy import nan

In [32]:
df_instrument_data_merged.query('startDate == None')

,instrument,nativeInstrument,nativeBaseSymbol,nativeQuoteSymbol,baseSymbol,quoteSymbol,underlying,quoteAsset,expiration,active,...,limits_price_max,limits_price_min,precision_amount,precision_base,precision_price,precision_quote,settleAsset,exchange,startDate,endDate


In [38]:
list(range(1 +1))

[0, 1]

In [40]:
df_instrument_data_merged.query('instrument == "ADAUSD_PERP"').iloc[0]

instrument                 ADAUSD_PERP
nativeInstrument           ADAUSD_PERP
nativeBaseSymbol                   ADA
nativeQuoteSymbol                  USD
baseSymbol                         ADA
quoteSymbol                        USD
underlying                         ADA
quoteAsset                         USD
expiration                            
active                            True
contractSize                        10
contractType                 perpetual
marginType                     inverse
limits_amount_max              2000000
limits_amount_min                    1
limits_cost_max                   None
limits_leverage_max               None
limits_leverage_min               None
limits_market_max                25000
limits_market_min                    1
limits_price_max                   202
limits_price_min                 0.017
precision_amount                   1.0
precision_base                     0.0
precision_price                 0.0001
precision_quote          